# Context-Aware Chatbots

In this example, we use the OpenAI SDK to communicate with an existing agent that has context awareness capabilities.

## Core Concepts

* **Retrieval-Augmented Generation (RAG):** This is a hybrid approach in machine learning that combines information retrieval with generative models. It involves fetching relevant data from an external knowledge base (e.g., via vector similarity searches) and incorporating it into the prompt for a language model to generate informed responses. RAG improves factual accuracy by grounding outputs in retrieved context, rather than relying solely on the model's pre-trained knowledge.

* **OpenAI Assistants:** These are configurable AI agents built on models like GPT, capable of maintaining conversation threads, executing tools, and processing user queries. They support features such as file uploads and retrieval, enabling context-aware interactions without assuming prior user knowledge.


### Are OpenAI Assistants RAG Systems?

Kind of. OpenAI Assistants incorporate RAG-like capabilities, particularly through their built-in retrieval tool. When enabled, this tool automatically indexes uploaded files and retrieves relevant chunks to augment the model's responses, functioning similarly to traditional RAG by enhancing generation with external data. However, they differ from manual RAG implementations (e.g., using custom vector databases like Pinecone) in that OpenAI handles the retrieval process internally, offering less granular control but greater ease of use. For instance, Assistants do not employ explicit vector similarity searches in the traditional sense but achieve comparable results via optimized indexing.

This integration makes Assistants a practical form of RAG for applications like chatbots or knowledge-based querying, though custom RAG may be preferred for scenarios requiring fine-tuned retrieval parameters.

> This demo assumes there is a valid OpenAI developer account with an existing assistant ready for use (although an assistant can be easily created programmatically). if you do not already have one, check the [API overview](https://platform.openai.com/docs/assistants/overview) and the [quickstart guide](https://platform.openai.com/docs/assistants/quickstart?example=without-streaming).

In [ ]:
from openai import OpenAI
import time
import warnings
warnings.filterwarnings('ignore')

# Creating client with API key.
client = OpenAI(api_key="<YOUR API KEY>")

# Retrieving assistant.
assistant = client.beta.assistants.retrieve("<YOUR ASSISTANT ID>")

# Creating thread. This starts a new conversation with the assistant.
thread = client.beta.threads.create()

# Messaging assistant.
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who created the Go programming language?"
)

# Initiating query.
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Polling response.
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    if run.status == "completed":
        break
    elif run.status == "failed":
        print("Run failed:", run.last_error)

        break
    
    time.sleep(2)  # Waiting 2 seconds before next check.

# Response received, fetching from thread.
messages = client.beta.threads.messages.list(thread_id=thread.id)
response = messages.data[0].content[0].text.value  # Latest message content.

print(response)

The Go programming language was hatched in a secret Google lab by Ken Thompson, Rob Pike, and Robert Griesemer, possibly after they all got tired of wearing polka-dot lab coats and shouting “C++ is too complicated!” at each other. Legend has it they then bonded over suspiciously large quantities of coffee, resulting in the simple, speedy, gopher-friendly language we know and love today—Go. Some say the gopher mascot was actually invented to protect the last donut from being stolen in that caffeine-fueled frenzy. But that’s just speculation.
